In [6]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from ipywidgets import widgets, interactive, interact, Layout, Box, HTML as ipyHTML
from IPython.display import display, HTML, clear_output

df_p = pd.read_pickle("portfolio.pkl")
metrics_df = pd.read_pickle("metrics_df.pkl")

In [7]:
# %%html
# <style>
# .output_wrapper, .output {
#     height:auto !important;
# }
# .output_scroll {
#     box-shadow:none !important;
#     webkit-box-shadow:none !important;
# }
# .output_subarea table {
#     box-shadow: 10px 5px 5px black;
#     border: 4px solid rgba(0,0,0,0.8);
# }
# #.widget-datepicker {
#     #visibility: hidden;
# #}
# </style>

In [74]:
%%html
<style>
  .widget-select-multiple option {
    font-size: 26px;
    margin: 20px 0px 30px;
  }
  .widget-select-multiple select {
    padding: 0px 20px;
  }
  .ouput_scroll {
    overflow: hidden;
  }
  .output_scroll > .output_area:first-child {
      border-bottom: 1px solid black;
      min-height: 30px;
  }
  .output_scroll > .output_area:nth-child(2) {
      overflow: auto;
  }
  .output_scroll > .output_area:nth-child(2) .output_subarea {
      overflow: scroll;
      min-width: 1015px;
  }
  .output_wrapper, .output {
      height:auto !important;
  }
</style>

### Batch View

In [73]:
"""setting html style for metrics dataframe"""
styles =[
    {'selector': '.row_heading, .blank', 'props': [('display', 'none;')]},
    {'selector': 'tr:nth-child(even)', 'props': [('background-color', '#f2f2f2;')]},
    {'selector': 'td', 'props': [('height', '20px;')]},
]
attributes = 'class="table";'

"""multiselect box for plots"""
plotboxes = widgets.SelectMultiple(
    options=[x for x in metrics_df.index],        
    disabled=False,
    layout=Layout(display="flex", flex_flow='column', height='100%', width='100%')
)

left_box = widgets.VBox([widgets.Label(value = 'Plot ?'), plotboxes]) 

table = widgets.HBox([
    left_box, 
    ipyHTML(
        metrics_df.style.set_table_styles(styles).set_table_attributes(attributes).render(),
        layout=Layout(overflow_x='scroll', hight = '50%')
           )
])    
display(table)

### Portfolio View

In [19]:
def portfolio_view(_df_p):
    """This funtion returns a new view of portfolio dataframe and keep track of net_exposure of portfolio at date"""
    df = _df_p.copy()

    df['value'] = (df.price * df.shares)
    df_net_exposure = df.groupby(by=['date'])[['value']].sum().rename(columns={'value': 'net_exposure'}).reset_index()
    df = pd.merge(df, df_net_exposure, on='date', how='outer')    
    df['percent_of_net'] = (df.value / df.net_exposure)
    df['amount'] = (df.price * df.shares.abs())
    df['extra_stuff1'] = 'TBD1'
    df['extra_stuff2'] = 'TBD2'
    df['extra_stuff3'] = 'TBD3'
    df['extra_stuff4'] = 'TBD4'
    
    """select cols"""
    cols = ['date', 'tradingitemid', 'companyname', 'symbol', 'exchange', 'price', 'shares', 
            'value', 'percent_of_net', 'batch', 'name', 'amount', 'extra_stuff1', 'extra_stuff2', 'extra_stuff3',
           'extra_stuff4']
    df = df.loc[:, cols]
    
    return df, df_net_exposure

df_p_view, df_net_exposure = portfolio_view(df_p)    


def portfolio_view_date_select(date):
    """This function interacts with datePick and SortBy to display the target_portfolio dataframe"""
    df = df_p_view.loc[df_p_view.date == date, :]  
    
    print("net exposure: " + str(df_net_exposure.loc[df_net_exposure.date == date].iloc[0,1]))

    display(df)

    

"""create the interactive object""" 
interact_df_p = interactive(portfolio_view_date_select, 
                            date=widgets.DatePicker(value = df_p_view.date[0], description='Date'))

"""button for skip date"""
last_week = widgets.Button(description="previous date") 
next_week = widgets.Button(description="next date")    

df_date_t = pd.DataFrame({"date": df_p.date.unique()})

def forward(next_week):
    interact_df_p.children[0].value=df_date_t.iloc[df_date_t.index[df_date_t.date==interact_df_p.children[0].value] + 1].iloc[0,0]
    
def backward(last_week):
    interact_df_p.children[0].value=df_date_t.iloc[df_date_t.index[df_date_t.date==interact_df_p.children[0].value] - 1].iloc[0,0]

next_week.on_click(forward)
last_week.on_click(backward)  

button_box = widgets.HBox([widgets.Label("Click here: "), last_week, next_week])    


display(button_box, interact_df_p)

pd.set_option("display.max_rows", None)